In [1]:
import numpy as np

In [2]:
# Download the dataset
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [3]:
train_size = 300
test_size = 50

In [4]:
x_train = x_train[:train_size]
y_train = y_train[:train_size]
x_test = x_test[:test_size]
y_test = y_test[:test_size]

In [5]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(300, 28, 28) (300,)
(50, 28, 28) (50,)


In [6]:
from skimage.transform import resize
x_train = resize(x_train, (x_train.shape[0], 224, 224), anti_aliasing=True)
x_test = resize(x_test, (x_test.shape[0], 224, 224), anti_aliasing=True)

/Users/alican/.virtualenvs/ml/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [7]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(300, 224, 224) (300,)
(50, 224, 224) (50,)


In [8]:
x_train = np.stack((x_train, x_train, x_train), axis=3)
x_test = np.stack((x_test, x_test, x_test), axis=3)

In [9]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(300, 224, 224, 3) (300,)
(50, 224, 224, 3) (50,)


In [10]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(300, 224, 224, 3) (300, 10)
(50, 224, 224, 3) (50, 10)


In [12]:
from keras.applications.mobilenet import MobileNet
MobileNet_extractor = MobileNet(weights='imagenet', include_top=False)

/Users/alican/.virtualenvs/ml/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [13]:
train_features = MobileNet_extractor.predict(x_train)
test_features = MobileNet_extractor.predict(x_test)

In [14]:
print(train_features.shape, y_train.shape)
print(test_features.shape, y_test.shape)

(300, 7, 7, 1024) (300, 10)
(50, 7, 7, 1024) (50, 10)


In [15]:
train_features = train_features.reshape(train_features.shape[0],-1)
test_features = test_features.reshape(test_features.shape[0],-1)

In [16]:
print(train_features.shape, y_train.shape)
print(test_features.shape, y_test.shape)

(300, 50176) (300, 10)
(50, 50176) (50, 10)


In [17]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [18]:
model = Sequential()
model.add(Dense(10, input_shape = (train_features.shape[1],), kernel_initializer='he_normal'))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                501770    
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 501,770
Trainable params: 501,770
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [20]:
model.fit(train_features, y_train, epochs=5, validation_data=(test_features, y_test))

Train on 300 samples, validate on 50 samples
Epoch 1/5
300/300 [==============================] - 1s 3ms/step - loss: 3.6623 - acc: 0.5833 - val_loss: 2.5284 - val_acc: 0.7800
Epoch 2/5
300/300 [==============================] - 0s 812us/step - loss: 2.5165 - acc: 0.8167 - val_loss: 1.4104 - val_acc: 0.8800
Epoch 3/5
300/300 [==============================] - 0s 823us/step - loss: 2.1603 - acc: 0.8633 - val_loss: 1.2898 - val_acc: 0.9200
Epoch 4/5
300/300 [==============================] - 0s 760us/step - loss: 1.3521 - acc: 0.9100 - val_loss: 1.5865 - val_acc: 0.8800
Epoch 5/5
300/300 [==============================] - 0s 755us/step - loss: 1.2802 - acc: 0.9100 - val_loss: 1.2895 - val_acc: 0.9200
